<i>## Comments will be provided using this format. Key takeaway: groups are encouraged to change the formatting, but not the structure. Groups are also allowed to create additional notebooks - for instance, create one notebook for data exploration, and one notebook for each preprocessing-modelling-evaluation pipeline -, but must strive to keep an unified style across notebooks.</i>

#### NOVA IMS / BSc in Data Science / Text Mining 2024/2025
### <b>Group Project: "Solving the Hyderabadi Word Soup"</b>
#### Notebook `Notebook Title`

#### Group:
- `Group member #1`
- `(...)`
- `Group member #5`

#### <font color='#BFD72'>Table of Contents </font> <a class="anchor" id='toc'></a> 
- [1. Data Understanding](#P1)
- [2. General Data Preparation](#P2) 
- [3. Multilabel Classification (Information Requirement 3311)](#P3)
    - [3.1 Specific Data Preparation](#P31)
    - [3.2 Model Implementation](#P32)
    - [3.3 Model Evaluation](#P3n)
- [4. Sentiment Analysis (Information Requirement 3312)](#P4)
    - [4.1 Specific Data Preparation](#P41)
    - [4.2 Model Implementation](#P42)
    - [4.3 Model Evaluation](#P43)
- [...]
- [N. Additional Tasks (Information Requirements 332n)](#Pn)
    - [N.1 Specific Data Preparation](#Pn1)
    - [N.2 Model Implementation](#Pn2)
    - [N.3 Model Evaluation](#Pn3)

<i>## Note that the notebook structure differs from the report: instead of following the CRISP-DM phases and then specifying the different problems inside the phases, the notebook is structured by problem, with the CRISP-DM phases being defined for each specific problem.

In [ ]:
## All imports must be concentrated on a cell that immediately follow the table of contents
import math
import time
import re
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize
from nltk.tokenize import PunktSentenceTokenizer
sent_tokenizer = PunktSentenceTokenizer()

#without truncation
pd.set_option('display.max_colwidth', None)

<font color='#BFD72F' size=5>1. Data Understanding</font> <a class="anchor" id="P1"></a>
  
[Back to TOC](#toc)

<i>## Imports.</i>

In [36]:
reviews=pd.read_csv('data/reviews.csv')
restaurants=pd.read_csv('data/restaurants.csv') 
restaurants = restaurants.drop(columns=['Links'])

## Restaurant data initial preprocessing


In [37]:
#Turning collunm cost to int
restaurants['Cost'] = restaurants['Cost'].str.replace(',', '').astype(int)

### solving null value in  timming


In [38]:
restaurants[restaurants['Timings'].isnull()] #there is a missing value in timings
#After visiting zomato website we took the timetable of this restaurant
restaurants.loc[restaurants['Timings'].isnull(), 'Timings'] = '12AM to 3:30pm, 7pm to 11pm (Mon-Sun)'

In [39]:
# Putting this 2 collumns in a list
restaurants['Collections'] = restaurants['Collections'].str.replace(r',\s+', ',', regex=True).str.split(',')
restaurants['Cuisines']=restaurants['Cuisines'].str.replace(r',\s+', ',', regex=True).str.split(',')

In [40]:
restaurants['N_collections'] = restaurants['Collections'].apply(lambda x: len(x) if type(x)==list else 0)

In [41]:
def capture_open_close_times(numbers_list):
    result = []
    for string in numbers_list:
        # Find all numbers in the string
        numbers = re.findall(r'\d+', string)
        if numbers:   
            # Capture the first (opening time) and determine closing time
            opening_time = numbers[0]
            if numbers[-1] in ('15', '30', '40'):
                closing_time = numbers[-2] if len(numbers) > 1 else None  # Use second-to-last if available
            else:
                closing_time = numbers[-1]  # Use last if not 15, 30, or 40

            # Transform closing time if it's 10, 11, or 12
            if closing_time == '10':
                closing_time = '22'
            elif closing_time == '11':
                closing_time = '23'
            elif closing_time == '12':
                closing_time = '24'

            # Transform opening time if it's 12, 1, 4, or 5
            elif opening_time == '1':
                opening_time = '13'
            elif opening_time == '4':
                opening_time = '16'
            elif opening_time == '5':
                opening_time = '17'

            # Append the transformed opening and closing times to the result
            result.append((opening_time, closing_time))

    return result

# Get a list of tuples (opening time, closing time)
open_close_times = capture_open_close_times(restaurants['Timings'])

# Unpack opening and closing times into separate lists
opening_times = [time[0] for time in open_close_times]
closing_times = [time[1] for time in open_close_times]

# Assign the lists to new columns in the DataFrame
restaurants['open time'] = opening_times
restaurants['closing time'] = closing_times
restaurants.drop('Timings', axis=1, inplace=True)

## reviews data initial preprocessing

In [42]:
reviews.drop_duplicates(inplace=True)

In [43]:
reviews[reviews.isna().any(axis=1)]

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures
2360,Amul,Lakshmi Narayana,NaN,5,0 Reviews,7/29/2018 18:00,0
5799,Being Hungry,Surya,NaN,5,"4 Reviews , 4 Followers",7/19/2018 23:55,0
6449,Hyderabad Chefs,Madhurimanne97,NaN,5,1 Review,7/23/2018 16:29,0
6489,Hyderabad Chefs,Harsha,NaN,5,1 Review,7/8/2018 21:19,0
7954,Olive Garden,ARUGULLA PRAVEEN KUMAR,NaN,3,"1 Review , 1 Follower",8/9/2018 23:25,0
8228,Al Saba Restaurant,Suresh,NaN,5,1 Review,7/20/2018 22:42,0
8777,American Wild Wings,NaN,NaN,NaN,NaN,NaN,0
8844,Domino's Pizza,Sayan Gupta,NaN,5,"2 Reviews , 2 Followers",8/9/2018 21:41,0
9085,Arena Eleven,NaN,NaN,NaN,NaN,NaN,0


In [44]:
reviews.drop([8777, 9085], axis=0, inplace=True)

In [45]:
reviews.Rating.value_counts()

5       3832
4       2373
1       1735
3       1193
2        684
4.5       69
3.5       47
2.5       19
1.5        9
Like       1
Name: Rating, dtype: int64

In [46]:
reviews[reviews['Rating']=='Like']

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures
7601,The Old Madras Baking Company,Dhanasekar Kannan,One of the best pizzas to try. It served with the fresh crust and the topping of veggies are fresh and the taste of the ingredients was awesome and it is fully overloaded with Cheese. I would like to recommend to try every Time I wager for pizza,Like,"12 Reviews , 21 Followers",5/18/2019 12:31,1


In [47]:
reviews.at[7601, 'Rating'] = 5

In [48]:
reviews['Rating'] = reviews['Rating'].astype(float)

In [49]:
# Extract the number of reviews, followers
reviews['N_reviews'] = reviews['Metadata'].str.extract(r'(\d+)\s+Review')
reviews['Followers'] = reviews['Metadata'].str.extract(r'(\d+)\s+Follower')

reviews['N_reviews'] = reviews['N_reviews'].astype('Int64')
reviews['Followers'] = reviews['Followers'].astype('Int64')
reviews=reviews.drop('Metadata', axis=1)

In [50]:
reviews['Time'] = pd.to_datetime(reviews['Time'])

reviews['Month'] = reviews['Time'].dt.month.astype(int)
reviews['Year'] = reviews['Time'].dt.year.astype(int)

reviews['Weekend'] = reviews['Time'].dt.weekday.apply(lambda x: 1 if x >= 5 else 0)

In [51]:
reviews['Hour'] = reviews['Time'].dt.hour

# Create the 'Post_Meal' column based on the hour ranges for lunch and dinner
# Lunch: 13-15, Dinner: 20-23
reviews['Post_Meal'] = reviews['Hour'].apply(lambda x: 1 if (13 <= x <= 15) or (20 <= x <= 23) else 0)
reviews.drop('Hour', axis=1, inplace=True)

In [ ]:
reviews["msg_len"] = reviews["Review"].map(lambda content : len(str(content)))
reviews["sents"] = reviews["Review"].map(lambda content :sent_tokenizer.tokenize(str(content)))
reviews["nr_sents"] = reviews["sents"].map(lambda content : len(content))

In [54]:
#restaurants.to_csv('data/restaurants_initial_preproc.csv', index=False)
#reviews.to_csv('data/reviews_initial_preproc.csv', index=False)